# Gaia colour magnitude diagram (CMD) for HOYS Targets

Welcome to the CMD notebook for the HOYS LCO Project.\
Please see the video tutorial.

### Notebook tips:
* ***Shift + Enter on a code cell/block to run it and advance to the next cell.***
* Can re-run blocks out of order as long as the variables are already there
* If python kernal crashses or out of memory, use the reset button above to reset the kernal.

In [ ]:
import time
import math
from matplotlib import *
from matplotlib.pyplot import *
import numpy as np
import pandas as pd
import astropy
from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
import numpy.ma as ma
import os
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widget
from IPython.display import display,clear_output


In [ ]:
#for lab
%matplotlib widget
#for notebook/slides
#%matplotlib notebook #for notebook/slides
rcParams.update({'figure.max_open_warning': 50})

In [ ]:
iso_table_1myr = pd.read_csv('gaia_edr3_iso_1myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_6myr = pd.read_csv('gaia_edr3_iso_6myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_7myr = pd.read_csv('gaia_edr3_iso_7myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_8myr = pd.read_csv('gaia_edr3_iso_8myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_9myr = pd.read_csv('gaia_edr3_iso_9myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_10myr = pd.read_csv('gaia_edr3_iso_10myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_20myr = pd.read_csv('gaia_edr3_iso_20myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)
iso_table_100myr = pd.read_csv('gaia_edr3_iso_100myr.txt',delim_whitespace=True,index_col=False,comment='#',skip_blank_lines=True)

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
target_sel=widget.Dropdown(
    options=[('NGC 1333', '03 29 02 +31 20 54'), 
             ('IC 348', '03 44 34 +32 09 48'), 
             ('Lambda – Ori Cluster','05 35 06 +09 56 00'),
             ('M42','05 35 17 -05 23 28'), 
             ('L 1641 N Cluster','05 36 19 -06 22 12'), 
             ('Sigma – Ori Cluster','05 38 45 -02 36 00'), 
             ('NGC 2068','05 46 46 +00 04 12'), 
             ('NGC 2244','06 31 55 +04 56 30'),
             ('MWSC 3274','20 11 13 +37 27 00'),
             ('P Cyg','20 17 47 +38 01 59'),
             ('Berkeley 86','20 20 12 +38 41 24'),
             ('IC 5070','20 51 00 +44 22 00'),
             ('IC 1396 A','21 36 35 +57 30 36'),
             ('IC 1396 N','21 40 42 +58 16 06'),
             ('NGC 7129','21 42 56 +66 06 12'),
             ('IC 5146','21 53 29 +47 16 01'),
            ],
    description='Target:',
)
display(target_sel)

In [ ]:
c = SkyCoord(target_sel.value, unit=(u.hourangle, u.deg))
print('querying Gaia catalogue for ',target_sel.label,' around coords:',c.to_string('hmsdms'))

radius = u.Quantity(0.5, u.deg)

Gaia.MAIN_GAIA_TABLE = "gaiaedr3.gaia_source" # Select early Data Release 3
#Gaia.MAIN_GAIA_TABLE = "gaiadr2.gaia_source"  # Reselect Data Release 2, default
Gaia.ROW_LIMIT=-1

j = Gaia.cone_search_async(c, radius,columns=[])

r = j.get_results()
r

In [ ]:
cluster_table_all=r.to_pandas()
pzero=0.5 #ignore <0 parallax
perr=3 #set min parallax error
g_min=18
cluster_table_all=cluster_table_all.query(f'parallax > {pzero} and parallax_error < {perr} and phot_g_mean_mag < {g_min}')

parallax=np.mean(cluster_table_all['parallax'])
print('mean parallax of entire region:',np.round(parallax,3),' mas')
print('mean distane to entire region:',np.round(1/parallax*1000,3),' pc')
dist=1/parallax*1000
distance_eDR3=pd.Series((1/cluster_table_all.parallax*10e2),name='distance_eDR3')
cluster_table_all=pd.concat([cluster_table_all,distance_eDR3],axis=1) #add distance column to each object
#calculate absolute G band magnitudes using distances
abs_phot_g_mean_mag_eDR3=pd.Series(cluster_table_all['phot_g_mean_mag']-5*np.log10(cluster_table_all['distance_eDR3']/10),name='abs_phot_g_mean_mag_eDR3')
cluster_table_all=pd.concat([cluster_table_all,abs_phot_g_mean_mag_eDR3],axis=1)
cluster_table_ms=cluster_table_all.query(f'parallax > 2 and parallax_error < {perr} and phot_g_mean_mag < {g_min}')

cluster_table_all

#### Plot CMD for entire region

In [ ]:
fig1, ax1 = subplots(1,1)
ax1.plot(cluster_table_all['bp_rp'],cluster_table_all['abs_phot_g_mean_mag_eDR3'],'g.',label='all query',markersize=1)
ax1.set_xlim(min(cluster_table_all['bp_rp'])-0.5,max(cluster_table_all['bp_rp'])+0.5)
ax1.set_ylim(max(cluster_table_all['abs_phot_g_mean_mag_eDR3'])+1,min(cluster_table_all['abs_phot_g_mean_mag_eDR3'])-1)
ax1.legend()
ax1.set_xlabel('BP-RP [mag]')
ax1.set_ylabel('G [mag]')
show()

#### Plot histogram of parallaxes and select window for cluster parallax

In [ ]:
def sel_par_range(prange=[0,6],wbins=0.05):
    global cluster_table_par
    cluster_table_par=cluster_table_all.query(f'parallax > {prange[0]} and parallax < {prange[1]}')
    figure()
    data=cluster_table_all['parallax'][cluster_table_all['parallax']<6]
    w = wbins
    n = math.ceil((max(data) - min(data))/w)
    hist(data,bins=n,label='all query')
    data2=cluster_table_par['parallax'][cluster_table_par['parallax']<6]
    n2 = math.ceil((max(data2) - min(data2))/w)
    hist(data2,bins=n2,label='cluster selection')
    xlabel('parallax [mas]')
    ylabel('count')
    legend()
    show()
    
interact(sel_par_range, prange=widget.FloatRangeSlider(value=[0,7],min=0,max=6,step=0.1,description='Parallax range [mas]:',
                                    readout_format='.1f',layout={'width': '600px'},style = {'description_width': 'initial'},
                                    continuous_update=False),
        wbins=widget.FloatSlider(value=0.05,min=0.01,max=1,step=0.01,description='bin width:',continuous_update=False))


#### plot cmd with parallax selection highlighted

In [ ]:
fig1, ax1 = subplots(1,1)
ax1.plot(cluster_table_all['bp_rp'],cluster_table_all['abs_phot_g_mean_mag_eDR3'],'g.',label='all query')
ax1.plot(cluster_table_par['bp_rp'],cluster_table_par['abs_phot_g_mean_mag_eDR3'],'r.',label='parallax selection')
ax1.set_xlim(min(cluster_table_all['bp_rp'])-0.5,max(cluster_table_all['bp_rp'])+0.5)
ax1.set_ylim(max(cluster_table_all['abs_phot_g_mean_mag_eDR3'])+1,min(cluster_table_all['abs_phot_g_mean_mag_eDR3'])-1)
ax1.legend()
ax1.set_xlabel('BP-RP [mag]')
ax1.set_ylabel('G [mag]')
show()

#### plot proper motions with parallax selection highlighted

In [ ]:
fig3, ax3 = subplots(1,1)
#ax3.plot(cluster_table_all['pmra'],cluster_table_all['pmdec'],'g.',label='all query')
#ax3.plot(cluster_table_par['pmra'],cluster_table_par['pmdec'],'r.',label='par  sel')
errorbar(cluster_table_all['pmra'],cluster_table_all['pmdec'], 
         xerr=cluster_table_all['pmra_error'], yerr=cluster_table_all['pmdec_error'], fmt=".", color="g",label='all query')
errorbar(cluster_table_par['pmra'],cluster_table_par['pmdec'], 
         xerr=cluster_table_par['pmra_error'], yerr=cluster_table_par['pmdec_error'], fmt=".", color="r",label='par sel')
xlim(-30,30)
ylim(-30,30)
ax3.legend()
ax3.set_xlabel('pmra [mas/yr]')
ax3.set_ylabel('pmdec [mas/yr]')
show()

##### zoom in on the above plot to set limits

In [ ]:
from scipy.stats import kde

pmdec_min, pmdec_max = ax3.get_ylim()
pmra_min, pmra_max = ax3.get_xlim()

cluster_table=cluster_table_par.query(f'pmra > {pmra_min} and pmra < {pmra_max} and pmdec > {pmdec_min} and pmdec < {pmdec_max}')

# select data
x = cluster_table['pmra']
y = cluster_table['pmdec']
 
# Evaluate a gaussian kde on a regular grid of nbins x nbins over data extents
nbins=300
k = kde.gaussian_kde([x,y])
xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
zi = k(np.vstack([xi.flatten(), yi.flatten()]))
 
# Make the plot
fig5, ax5 = subplots(1,1)
ax5.plot(cluster_table['pmra'],cluster_table['pmdec'],'k.',label='all query',alpha=0.1)
ax5.pcolormesh(xi, yi, zi.reshape(xi.shape),norm=colors.PowerNorm(gamma=5),shading='auto',cmap='Spectral_r')

ax5.set_xlabel('pmra [mas/yr]')
ax5.set_ylabel('pmdec [mas/yr]')
show()

##### zoom in again on the above KDE plot to set limits for final selection

#### plot proper motions with selections

In [ ]:
pmdec_min, pmdec_max = ax5.get_ylim()
pmra_min, pmra_max = ax5.get_xlim()

cluster_table=cluster_table_par.query(f'pmra > {pmra_min} and pmra < {pmra_max} and pmdec > {pmdec_min} and pmdec < {pmdec_max}')

fig1, ax1 = subplots(1,1)
ax1.plot(cluster_table_all['pmra'],cluster_table_all['pmdec'],'g.',label='all query')
ax1.plot(cluster_table['pmra'],cluster_table['pmdec'],'r.',label='par and pm sel')

#ax1.errorbar(cluster_table_all['pmra'],cluster_table_all['pmdec'], 
#         xerr=cluster_table_all['pmra_error'], yerr=cluster_table_all['pmdec_error'], fmt=".", color="g",label='all query')
#ax1.errorbar(cluster_table['pmra'],cluster_table['pmdec'], 
#         xerr=cluster_table['pmra_error'], yerr=cluster_table['pmdec_error'], fmt=".", color="r",label='par and pm sel')
ax1.set_xlim(-30,30)
ax1.set_ylim(-30,30)
ax1.legend()
ax1.set_xlabel('pmra [mas/yr]')
ax1.set_ylabel('pmdec [mas/yr]')
show()

#### plot CMD with parallax and proper motion selections

In [ ]:
fig1, ax1 = subplots(1,1)
ax1.plot(cluster_table_all['bp_rp'],cluster_table_all['abs_phot_g_mean_mag_eDR3'],'g.',label='all query')
ax1.plot(cluster_table['bp_rp'],cluster_table['abs_phot_g_mean_mag_eDR3'],'r.',label='cluster parallax and pm selection')
ax1.set_xlim(min(cluster_table_all['bp_rp'])-0.5,max(cluster_table_all['bp_rp'])+0.5)
ax1.set_ylim(max(cluster_table_all['abs_phot_g_mean_mag_eDR3'])+1,min(cluster_table_all['abs_phot_g_mean_mag_eDR3'])-1)
ax1.legend()
ax1.set_xlabel('BP-RP [mag]')
ax1.set_ylabel('G [mag]')
show()

In [ ]:
parallax=np.mean(cluster_table['parallax'])
print('mean parallax of selected cluster members:',np.round(parallax,3),' mas')
print('mean distane to selected cluster members:',np.round(1/parallax*10e2,3),' pc')
dist=1/parallax*10e2

#### Overplot isochrones

In [ ]:
#need to add selections for which isochrones to plot

fig2, ax2 = subplots(1,1)
ax2.plot(cluster_table['bp_rp'],cluster_table['abs_phot_g_mean_mag_eDR3'],'r.',label='Cluster')
ax2.plot(cluster_table_ms['bp_rp'],cluster_table_ms['abs_phot_g_mean_mag_eDR3'],'k.',label='Main Sequence',markersize=1)

ax2.plot(iso_table_100myr['G_BPmag']-iso_table_100myr['G_RPmag'],iso_table_100myr['Gmag'],'k-',label='100 Myr')
ax2.plot(iso_table_10myr['G_BPmag']-iso_table_10myr['G_RPmag'],iso_table_10myr['Gmag'],'b-',label='10 Myr')
ax2.plot(iso_table_1myr['G_BPmag']-iso_table_1myr['G_RPmag'],iso_table_1myr['Gmag'],'g-',label='1 Myr')

ax2.set_xlim(min(cluster_table['bp_rp'])-0.5,max(cluster_table['bp_rp'])+0.5)
ax2.set_ylim(max(cluster_table['abs_phot_g_mean_mag_eDR3'])+1,min(cluster_table['abs_phot_g_mean_mag_eDR3'])-1)
ax2.legend()
ax2.set_xlabel('BP-RP [mag]')
ax2.set_ylabel('G [mag]')
show()

In [ ]:
close('all')